In [1]:
import os
import re
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import hashlib
import random
from plotly.subplots import make_subplots
from datetime import timedelta, datetime as dt
from pandas.tseries.offsets import MonthEnd
from string import ascii_uppercase
from math import floor, ceil, isnan
from itertools import chain

from salesforce_api import Salesforce

# Get Salesforce Data

In [4]:
client = Salesforce(
    username=os.getenv("SFDC_USER"),
    password=os.getenv("SFDC_PW"),
    security_token=os.getenv("SFDC_TOKEN")
)

https://login.salesforce.com/services/Soap/c/48.0


In [5]:
# Replace with your column names
columns = [
    "Id", "AccountId", "CurrencyIsoCode", "Contract_Start_Date__c",
    "Contract_Expiration_Date__c", "Contract_Term_of_mo__c", "Subscription_Amount__c",
    "StageName"
]

In [6]:
# Query opportunity data - Adjust to fit your needs
opportunities_raw = client.sobjects.query(
    f"""
    SELECT {', '.join(columns)} 
    FROM Opportunity 
    WHERE StageName IN ('Closed Won','Renewal')
    AND Contract_Start_Date__c >= 2020-01-01
    """
)

In [ ]:
# Convert to data frame
opportunities = pd.DataFrame(opportunities_raw).drop("attributes", axis=1)

In [8]:
# Clean up column names
opportunities.columns = [
    re.sub("__c", "", col.lower()) for col in opportunities.columns
]

# Transform

## Ensure correct data
- Convert currency to USD
- Filter out zero month contract length and zero revenue

In [13]:
exchange_rates = pd.DataFrame(
    (
        ["EUR", "GBP", "INR", "MXN", "MGA", "AUS", "CAN", "CAD", "USD", "ARS"],
        [1.1344, 1.3514, .0135, .0489, .0003, .7263, .7857, .7857, 1, .0091]
    )
).transpose().rename(columns={
    0: "currencyisocode",
    1: "exchange_rate"
})

In [14]:
opportunities = opportunities.loc[
    (opportunities["contract_term_of_mo"] > 0) &
    (opportunities["subscription_amount"] > 0), :
].merge(exchange_rates, on="currencyisocode", how="left").assign(
    converted_contract_value=lambda row: (
        row["subscription_amount"]*row["exchange_rate"]),
    subscription_mrr=lambda row: row["converted_contract_value"] /
        row["contract_term_of_mo"]
)

## Create date fields for comparison

In [15]:
# Ensure datetime format for expiration date and add 1 day
opportunities["contract_expiration_date"] = pd.to_datetime(
    opportunities["contract_expiration_date"],
    format="%Y-%m-%d"
)+timedelta(days=1)
# Create field containing the first day of the starting month
opportunities["contract_start_month_bom"] = pd.to_datetime(
    opportunities["contract_start_date"],
    format="%Y-%m-%d"
).apply(lambda x: x.replace(day=1))
# Create field containing the last day of the starting month
opportunities["contract_start_month_eom"] = pd.to_datetime(
    opportunities["contract_start_date"],
    format="%Y-%m-%d"
) + MonthEnd(1)
# Create field containing the first day of the ending month
opportunities["contract_end_month_bom"] = pd.to_datetime(
    opportunities["contract_expiration_date"],
    format="%Y-%m-%d"
).apply(lambda x: x.replace(day=1))
# Create field containing the last day of the ending month
opportunities["contract_end_month_eom"] = pd.to_datetime(
    opportunities["contract_expiration_date"],
    format="%Y-%m-%d"
) + MonthEnd(1)

## Create data frame with start and end dates for all possible months

In [17]:
months = pd.DataFrame(
    [
        list(pd.date_range(
            start=opportunities["contract_start_month_bom"].min(), 
            end=opportunities["contract_start_month_eom"].max(), 
            freq='MS'
        )),
        list(pd.date_range(
            start=opportunities["contract_start_month_bom"].min(), 
            end=opportunities["contract_start_month_eom"].max(), 
            freq='M'
        )),
    ]
).transpose()
months.columns = ["period_start_date", "period_end_date"]

## Cartesian join with months and opportunities

In [20]:
opportunity_months = opportunities.assign(key=1).merge(
    months.assign(key=1), 
    on="key"
).drop("key", axis=1)

## Create subscription status field

In [22]:
opportunity_months = opportunity_months.assign(
    months_after_contract_start=lambda row: np.round((row["period_start_date"] - row["contract_start_month_bom"])/np.timedelta64(1, 'M')),
    months_after_contract_end=lambda row: np.round((row["period_start_date"] - row["contract_end_month_bom"])/np.timedelta64(1, 'M')),
    subscription_status=lambda row: np.where(
        row["period_start_date"] < row["contract_start_month_bom"],
        "Pre-Active",
        np.where(
            (row["period_start_date"] >= row["contract_end_month_bom"]) & (row["months_after_contract_end"] == 0),
            "Churn",
            np.where(
                row["period_start_date"] > row["contract_end_month_bom"],
                "Post-Churn",
                "Active"
            )
        )
    )
)

## Zero out revenue for non-active periods

In [23]:
opportunity_months = opportunity_months[
    [
        "id", "accountid", "period_end_date", "subscription_status", "subscription_mrr"
    ]
].assign(
    subscription_mrr=lambda row: np.where(
        row["subscription_status"].isin(["Pre-Active", "Churn", "Post-Churn"]),
        0,
        row["subscription_mrr"]
    )
)

## Group to the account-period level

In [28]:
cohort_df = opportunity_months.loc[
    pd.to_datetime(opportunity_months["period_end_date"]) <= dt.today()
].groupby(
    ["accountid", "period_end_date"]
).agg({
    "subscription_mrr": "sum"
}).reset_index().astype({"subscription_mrr": "float"})

## Assign a "Group Index" to identify cohorts

In [29]:
# Get the cumulative sum for each account
cohort_df["cume_sum"] = cohort_df.groupby(["accountid"])["subscription_mrr"].cumsum()

# Remove any account that doesn't have revenue
cohort_df = cohort_df.loc[cohort_df["cume_sum"] > 0, :]

account_list = cohort_df["accountid"].unique()

for account in account_list:
    # filter to one account
    values = cohort_df.loc[
        cohort_df["accountid"]== account, :
    ].values.tolist()

    group_list = list()
    # Group index to assign that denotes a cohort
    idx = 0
    # Loop over each account period
    for i, row in enumerate(values):
        # If it is the first row then append the index to group_list
        if i == 0:
            group_list.append(idx)
        else:
            # If the mrr (row[2]) is positive then append the index to group_list
            if row[2] > 0:
                group_list.append(idx)
            
            # If the mrr is zero then append the index and increment by 1
            else:
                group_list.append(idx)
                idx += 1
    
    # Assign the list of indices to the account
    cohort_df.loc[
        cohort_df["accountid"] == account, "group_index"
    ] = group_list

## Assign a cohort and the starting revenue

In [31]:
agg_cohort_df = cohort_df.groupby(
    ["accountid", "group_index"]
).agg({
    "period_end_date": "first",
    "subscription_mrr": "first"
}).reset_index().rename(
    columns={
        "period_end_date": "cohort_period_end_date",
        "subscription_mrr": "starting_mrr"
    })
cohort_df = cohort_df.merge(
    agg_cohort_df,
    on=["accountid", "group_index"]
)

## Calculate the number of periods since joining the cohort

In [33]:
cohort_df["periods_since_joining_cohort"] = cohort_df.groupby(
    ["accountid", "group_index"]).cumcount()

## Pivot the data

In [35]:
cohort_pivot_mrr = cohort_df.loc[
    cohort_df["subscription_mrr"] > 0, :
].pivot_table(
    index="cohort_period_end_date",
    columns="periods_since_joining_cohort",
    values="subscription_mrr",
    aggfunc="sum"
).reset_index().sort_values("cohort_period_end_date")

# Visualizing Your Cohorts

In [38]:
date_periods = 25

x = np.array(list(range(date_periods)))
y = cohort_pivot_mrr.tail(date_periods)["cohort_period_end_date"].values

## The Easy Way

In [43]:
z_mrr = cohort_pivot_mrr.tail(date_periods).loc[:, x].values
z_starting = cohort_pivot_mrr.tail(date_periods).loc[:, x].values
z_starting = [z[0] for z in z_starting]
z_ret = [z[0]/z[1] for z in zip(z_mrr, z_starting)]

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x=x,
        y=y,
        z=z_ret,
        hoverongaps=False,
        texttemplate="%{z:.0%}",
        colorscale="greens",
        showscale=False,
        showlegend=False,
        zmin=0.5,
        zmid=1,
        zmax=1.5,
    )
)
fig.update_traces(
    showscale=False
)
fig.update_coloraxes(
    showscale=False
)
fig.update_layout(
    plot_bgcolor="white",
    uniformtext_minsize=6,
    uniformtext_mode='hide',
    margin_pad=5,
    font={
        "size": 10
    },
    title={
        "text": "Last 25 Periods MRR",
        "font": {"size": 20},
        "x": 0.5
    }
)
fig.update_yaxes(
    type="category",
    autorange="reversed",
    tickfont={"size": 10}
)
fig.update_xaxes(
    type="category",
    tickfont={"size": 10}
)

## The Hard Way

In [86]:
annotation_dict = {
    "2021-05-31": {"color": "purple", "adj_y_start": 0, "adj_y_end": 1.8},
    "2021-10-31": {"color": "red", "adj_y_start": 0.5, "adj_y_end": 2.3}
}

z_mrr = cohort_pivot_mrr.tail(date_periods).loc[:, x].values

fig = make_subplots(
    rows=len(y), cols=1,
    row_heights=[
        2 if el in annotation_dict.keys() else 1 for el in y
    ],
    vertical_spacing=0,
    shared_xaxes=True,
    shared_yaxes=True,
    column_titles=x.tolist(),
    x_title="Periods Since Joining Cohort"
)

for i, cohort in enumerate(y):
    fig.add_trace(
        go.Heatmap(
            x=x,
            y=[y[i]]*len(x),
            z=z_mrr[i]/z_mrr[i][0],
            hoverongaps=False,
            texttemplate="%{z:.0%}",
            hovertemplate="Cohort End Date: <b>%{y}</b> <br>"
            "Months Since Joining Cohort: <b>%{x}</b> <br>"
            "Retained MRR: <b>%{z:.2%}</b> <br>"
            "<extra></extra>",
            colorscale="greens",
            zmin=0.5,
            zmid=1,
            zmax=1.5,
            showscale=False,
            showlegend=False,
        ),
        row=i+1,
        col=1
    )

fig.update_yaxes(
    type="category",
    tickfont={"size": 10},
)
fig.update_xaxes(
    type="category",
    tickfont={"size": 10}
)

fig.update_layout(
    plot_bgcolor="white",
    uniformtext_minsize=6,
    uniformtext_mode='hide',
    margin_pad=5,
    font={
        "size": 10
    },
    title={
        "text": "Last 25 Periods MRR",
        "font": {"size": 20},
        "x": 0.5
    }
)

for k, v in annotation_dict.items():
    x_start = -0.09
    x_end = len(x) - np.where(y == k)[0][0]
    y_start = len(x) - np.where(y == k)[0][0] - \
        annotation_dict.get(k).get("adj_y_start")
    y_end = len(x) - np.where(y == k)[0][0] - \
        annotation_dict.get(k).get("adj_y_end")

    fig.add_shape(
        type="rect",
        xref="paper",
        yref="paper",
        x0=x_start,
        x1=x_end / len(x),
        y0=y_start / len(x),
        y1=y_end / len(x),
        line=dict(
            color=annotation_dict.get(k).get("color"),
            width=2,
        ),
    )

fig.add_annotation(
    text="""
    The <b><span style='color: purple'>May 2021</span></b> cohort has continued to show<br>
    strong retention as those customers reach<br>
    the 2 year mark.<br>
    <br>
    Conversely, the <b><span style='color: red'>October 2021</span></b> cohort saw a large reduction in<br>
    revenue in month 1. Efforts have been made to upsell remaining<br>
    customers but so far only half of initial revenue has been recovered.
    """,
    xref="paper", yref="paper",
    x=1.05, y=0.1,
    font_size=12,
    align="left",
    showarrow=False
)

fig.show()

## Write to an HTML file if you want

In [42]:
fig.write_html(
    "cohort_chart_output.html",
    full_html=False,
    include_plotlyjs='cdn'
)